In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras

# Plotting imports
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
%warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

from numpy.random import seed
seed(1)
# Allows us to see more information regarding the DataFrame
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

2022-08-11 20:30:41.784067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-11 20:30:41.784100: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
UsageError: Line magic function `%warnings.filterwarnings("ignore")` not found.


In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [3]:
final_store_cat_df = pd.read_csv('final_store_cat_df.csv')

In [4]:
final_store_cat_df.drop(('Unnamed: 0'), axis = 1, inplace = True)

In [5]:
data = pd.DataFrame(final_store_cat_df)

SARIMA uses past values but also takes into account any seasonality patterns. It brings in seasonality as a parameter.

In [6]:
#splitting in test and training data
data_training = data[:-31]
data_test = data[len(data)-31:]

In [7]:
data_training

,date,CA_1_FOODS,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_2_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_3_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,...,TX_3_HOUSEHOLD,WI_1_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_2_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,WI_3_FOODS,WI_3_HOBBIES,WI_3_HOUSEHOLD
0,2011-01-29,3239.0,556.0,542.0,2193.0,538.0,763.0,3446.0,550.0,743.0,...,503.0,1581.0,615.0,508.0,1615.0,190.0,451.0,3028.0,278.0,732.0
1,2011-01-30,3137.0,498.0,520.0,1921.0,397.0,728.0,3535.0,430.0,862.0,...,502.0,1327.0,443.0,424.0,1433.0,127.0,362.0,3106.0,356.0,736.0
2,2011-01-31,2008.0,415.0,393.0,1289.0,368.0,464.0,2701.0,438.0,646.0,...,370.0,977.0,323.0,262.0,1586.0,113.0,319.0,2543.0,248.0,526.0
3,2011-02-01,2258.0,392.0,401.0,1540.0,350.0,434.0,3064.0,424.0,744.0,...,320.0,935.0,137.0,179.0,2013.0,124.0,385.0,2596.0,194.0,421.0
4,2011-02-02,2032.0,268.0,330.0,1278.0,296.0,368.0,2761.0,364.0,692.0,...,201.0,2.0,0.0,0.0,967.0,58.0,150.0,1854.0,74.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,2016-03-20,4240.0,493.0,1464.0,1406.0,215.0,679.0,4755.0,688.0,2332.0,...,1098.0,3508.0,462.0,1033.0,4159.0,292.0,1206.0,3120.0,273.0,923.0
1878,2016-03-21,2766.0,532.0,944.0,2245.0,342.0,909.0,4003.0,508.0,1728.0,...,931.0,2180.0,413.0,656.0,3309.0,219.0,974.0,2234.0,171.0,690.0
1879,2016-03-22,2598.0,440.0,738.0,2514.0,286.0,845.0,3761.0,439.0,1503.0,...,779.0,2197.0,302.0,534.0,2984.0,222.0,862.0,2185.0,171.0,611.0
1880,2016-03-23,2554.0,454.0,762.0,2177.0,387.0,758.0,3666.0,540.0,1445.0,...,907.0,2065.0,223.0,463.0,2866.0,229.0,831.0,1898.0,199.0,530.0


In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#defining the evaluation metrics
#from: https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [9]:
from pmdarima import auto_arima

In [10]:
#dictionary to store results
Optimization_Model = {}


#Loop to optimize model for all series
for i in range(1,7):
    
    #defining the time series
    series = pd.Series(data.iloc[:, i])
    series.index = data["date"]
    
    #Checking for zero days as these dont work in an ETS model with multiplicativ components, 
    #and turning thm into the mean of the day before and after
    for day in range(0, len(series)):
        if series[day] <= 0:
            series[day] = ( series[day-1] + series[day+1] ) /2
    
    series_training = pd.Series(data_training.iloc[:, i])
    series_training.index = data_training["date"]
    
    series_test = pd.Series(data_test.iloc[:, i])
    series_test.index = data_test["date"]
     
    print(series.name)

    #Running the SARIMAX Model
    
    model_sarima = auto_arima(y=series_training, 
                       seasonal=True,
                       m=7,#weekly seasonality has the length 7
                       stepwise=True) #uses the  stepwise algorithm outlined in Hyndman and Khandakar (2008) 
    
    model_arima = auto_arima(y=series_training,
                       stepwise=True) 
    
    print(model_sarima.summary())
    print(model_arima.summary())
    
    #Creating model forecast
    forecast_point_sarima = model_sarima.predict(n_periods=31,return_conf_int=False)
    forecast_point_arima = model_arima.predict(n_periods=31,return_conf_int=False)
    
    #Extract fitted values
    fitted_point_sarima = model_sarima.predict_in_sample(n_periods=len(series_training),return_conf_int=False)
    fitted_point_arima = model_arima.predict_in_sample(n_periods=len(series_training),return_conf_int=False)
    
    #Metrics 
    SARIMA_fit = eval_metrics(series_training,fitted_point_sarima)
    SARIMA_forecast = eval_metrics(series_test[0:31],forecast_point_sarima[0:31])
    
    ARIMA_fit = eval_metrics(series_training,fitted_point_arima)
    ARIMA_forecast = eval_metrics(series_test[0:31],forecast_point_arima[0:31])
    
    print("Goodness of Fit SARIMA:",SARIMA_fit)
    print("Forecasting accuracy SARIMA:",SARIMA_forecast)
    
    print("Goodness of Fit ARIMA:",ARIMA_fit)
    print("Forecasting accuracy ARIMA:",ARIMA_forecast)
    
    #choosing Model
    if SARIMA_forecast[2] >= ARIMA_forecast[2]:
        Optimization_Model[series.name] = {'order': model_sarima.order, 
                                           'seasonal order': model_arima.seasonal_order,
                                          'mae': SARIMA_forecast[0],
                                          'rmse': SARIMA_forecast[1],
                                          'r2': SARIMA_forecast[2] }
        
        forecast = pd.Series(forecast_point_sarima,index=series_test.index)
        fit = pd.Series(fitted_point_sarima,index=series_training.index)
        
    else:
        Optimization_Model[series.name] = {'order': model_arima.order, 
                                           'seasonal order': 'None',
                                          'mae': ARIMA_forecast[0],
                                          'rmse': ARIMA_forecast[1],
                                          'r2': ARIMA_forecast[2] }
        
        forecast = pd.Series(forecast_point_arima,index=series_test.index)
        fit = pd.Series(fitted_point_arima,index=series_training.index)
        
    

CA_1_FOODS
                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                 1882
Model:             SARIMAX(5, 1, 0)x(2, 0, 0, 7)   Log Likelihood              -13635.347
Date:                           Thu, 11 Aug 2022   AIC                          27286.695
Time:                                   20:36:44   BIC                          27331.011
Sample:                                        0   HQIC                         27303.017
                                          - 1882                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.4040      0.020    -20.498      0.000      -0.443      -0.365
ar.L2         -0.2984      0.022

/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning:

Error fitting  ARIMA(2,1,0)(1,0,1)[7] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 508, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/home/studio-lab-user/.conda/envs/d

                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                 1882
Model:             SARIMAX(5, 1, 0)x(2, 0, 0, 7)   Log Likelihood              -11344.621
Date:                           Thu, 11 Aug 2022   AIC                          22705.241
Time:                                   20:43:55   BIC                          22749.558
Sample:                                        0   HQIC                         22721.564
                                          - 1882                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.5549      0.020    -27.846      0.000      -0.594      -0.516
ar.L2         -0.4179      0.024    -17.749

/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning:

Error fitting  ARIMA(2,1,1)(1,0,2)[7] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 508, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/home/studio-lab-user/.conda/envs/d

                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 1882
Model:             SARIMAX(3, 1, 0)x(2, 0, [1], 7)   Log Likelihood              -12991.884
Date:                             Thu, 11 Aug 2022   AIC                          25997.768
Time:                                     20:48:30   BIC                          26036.545
Sample:                                          0   HQIC                         26012.050
                                            - 1882                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.5249      0.011    -48.544      0.000      -0.546      -0.504
ar.L2         -0.3525      

/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning:

Error fitting  ARIMA(3,1,1)(1,0,1)[7] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pmdarima/arima/arima.py", line 508, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/home/studio-lab-user/.conda/envs/d

                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 1882
Model:             SARIMAX(1, 1, 2)x(1, 0, [1], 7)   Log Likelihood              -11296.219
Date:                             Thu, 11 Aug 2022   AIC                          22604.439
Time:                                     21:00:13   BIC                          22637.676
Sample:                                          0   HQIC                         22616.681
                                            - 1882                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2694      0.093     -2.892      0.004      -0.452      -0.087
ma.L1         -0.4551      

In [12]:
forecast_point_arima

array([959.20743157, 916.39858439, 826.38323388, 844.60921198,
       842.72424869, 761.82397954, 805.11725079, 918.75988635,
       914.50431542, 849.52570273, 841.60599548, 821.14680093,
       777.82656822, 817.13860174, 901.79075372, 909.01935392,
       864.26128052, 838.67998947, 810.75632425, 786.48012974,
       824.39995949, 891.82211751, 905.47259726, 871.88748053,
       837.34474361, 805.5009924 , 791.78122854, 828.56177195,
       885.98767365, 902.86335592, 875.72221766])

In [13]:
from numpy import asarray
from numpy import savetxt

In [14]:
# save array to file

CA_1_FOODS_yhat_arima = asarray(forecast_point_arima)

In [15]:
# save to csv file
savetxt('CA_1_FOODS_arima.csv',CA_1_FOODS_yhat_arima, delimiter=',')

In [16]:
forecast_point_sarima

array([ 904.82711558, 1353.03443564, 1322.48437779,  823.28882123,
        734.28322636,  737.60537268,  729.2487461 ,  893.46872492,
       1347.59912024, 1315.3867662 ,  817.01390888,  727.95179147,
        731.30442506,  722.94509788,  887.04614332, 1340.84038747,
       1308.65200511,  810.64729406,  721.65098747,  725.00114077,
        716.6479903 ,  880.62779923, 1334.08678467, 1301.92218276,
        804.28539275,  715.35483577,  718.70251401,  710.35553477,
        874.21419701, 1327.33817136, 1295.19733234])

In [17]:
# save array to file

CA_1_FOODS_yhat_sarima = asarray(forecast_point_sarima)

In [18]:
# save to csv file
savetxt('CA_1_FOODS_sarima.csv',CA_1_FOODS_yhat_sarima, delimiter=',')